In [1]:
import ee
import geemap
# Authenticate with Earth Engine
ee.Authenticate()
ee.Initialize()

In [2]:
# Define the asset ID of the FeatureCollection containing your polygons
fc_asset_id = 'projects/ee-chesangirine55/assets/TaitaTaveta2'

# Load the FeatureCollection
fc = ee.FeatureCollection(fc_asset_id)

# Get information about the FeatureCollection
fc_info = fc.getInfo()

# Print information about the FeatureCollection
#print("Feature Collection Information:")
#print(fc_info)


In [3]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define the asset ID of the FeatureCollection containing your polygon
fc_asset_id = 'projects/ee-chesangirine55/assets/TaitaTaveta2'

# Load the FeatureCollection
fc = ee.FeatureCollection(fc_asset_id)

# Get the first feature (assuming there's only one polygon in the collection)
#polygon = fc.first()

# Get the geometry of the polygon
geometry = fc.geometry()

# Define date range
start_date = '2022-04-01'
end_date = '2022-05-05'

# Load Sentinel-2 image collection
s2 = ee.ImageCollection('COPERNICUS/S2')
filtered_s2 = s2.filterBounds(geometry)\
                 .filterDate(start_date, end_date)\
                 .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 15)

# Clip images to the polygon
clipped_s2 = filtered_s2.map(lambda image: image.clip(geometry))

# Print the number of images in the filtered and clipped collection
print("Number of images:", clipped_s2.size().getInfo())

# Visualize the first 5 images in the collection
Map = geemap.Map()
Map.centerObject(geometry, zoom=8)

# Get first 5 images
images = clipped_s2.toList(5)

for i in range(5):
    image = ee.Image(images.get(i))
    Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Image ' + str(i+1))

#Map.addLayer(geometry, {}, 'Polygon')
Map.addLayerControl()
Map


C:\Users\ADMIN\miniconda3i\envs\ee\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


Number of images: 6


Map(center=[-3.4355456688820065, 38.41859110288993], controls=(WidgetControl(options=['position', 'transparent…

In [4]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define the asset ID of the FeatureCollection containing your polygon
fc_asset_id = 'projects/ee-chesangirine55/assets/kamatiraboundary'

# Load the FeatureCollection
fc = ee.FeatureCollection(fc_asset_id)

# Get the geometry of the polygon
geometry = fc.geometry()

# Define date range
start_date = '2022-04-01'
end_date = '2022-05-05'

# Load Sentinel-2 image collection
s2 = ee.ImageCollection('COPERNICUS/S2')
filtered_s2 = s2.filterBounds(geometry)\
                 .filterDate(start_date, end_date)\
                 .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 15)

# Clip images to the polygon
clipped_s2 = filtered_s2.map(lambda image: image.clip(geometry))

# Get the first image from the collection
first_image = clipped_s2.first()

# Visualize the original image (for reference)
Map = geemap.Map()
Map.centerObject(geometry, zoom=8)
Map.addLayer(first_image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'First Image (RGB)')

# ---------------------- Clustering (K-Means) ----------------------

# Sample the pixels from the selected image for clustering
sampled = first_image.sample(
    region=geometry,  # The region of interest (polygon)
    scale=10,         # Sampling scale in meters
    numPixels=5000    # Number of pixels to sample for clustering
)

# Perform K-Means clustering with 3 clusters
num_clusters = 3
clusterer = ee.Clusterer.wekaKMeans(num_clusters).train(sampled)
clustered = first_image.cluster(clusterer)

# Define visualization parameters for the clustered image
cluster_vis = {
    'min': 0,
    'max': num_clusters - 1,  # Since we have 3 classes (0, 1, 2)
    'palette': ['red', 'green', 'blue']
}

# Add the clustering result to the map
Map.addLayer(clustered, cluster_vis, 'K-Means Clustering (3 Classes)')

# Add the original image to the map (RGB)
Map.addLayer(first_image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Original Image (RGB)')

# Add Layer Control
Map.addLayerControl()

# Display the map
Map


Map(center=[1.269023644853747, 35.19263182187535], controls=(WidgetControl(options=['position', 'transparent_b…

In [5]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define the asset ID of the FeatureCollection containing your polygon
fc_asset_id = 'projects/ee-chesangirine55/assets/kamatiraboundary'

# Load the FeatureCollection
fc = ee.FeatureCollection(fc_asset_id)

# Get the geometry of the polygon
geometry = fc.geometry()

# Define date range
start_date = '2024-10-01'
end_date = '2025-01-20'

# Load Sentinel-2 image collection
s2 = ee.ImageCollection('COPERNICUS/S2')
filtered_s2 = s2.filterBounds(geometry)\
                 .filterDate(start_date, end_date)\
                 .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 5)

# Clip images to the polygon
clipped_s2 = filtered_s2.map(lambda image: image.clip(geometry))

# Get the first image from the collection
first_image = clipped_s2.first()

# Visualize the original image (for reference)
Map = geemap.Map()
Map.centerObject(geometry, zoom=8)
Map.addLayer(first_image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Original Image (RGB)')

# ---------------------- Upscaling to 1-meter resolution ----------------------

# Reproject to 1-meter resolution (upscale)
upscaled_image = first_image.reproject(crs='EPSG:4326', scale=1)

# Visualize the upscaled image at 1-meter resolution
Map.addLayer(upscaled_image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Upscaled Image (1-meter)')

# Add Layer Control
Map.addLayerControl()

# Display the map
Map


Map(center=[1.269023644853747, 35.19263182187535], controls=(WidgetControl(options=['position', 'transparent_b…

In [6]:
import ee

# Initialize Earth Engine
ee.Initialize()

# Load the FeatureCollection containing the polygons
fc = ee.FeatureCollection('projects/ee-chesangirine55/assets/kamatiraboundary')

# Get the list of features from the FeatureCollection as a client-side list
features_list = fc.toList(fc.size())

# Iterate over the features and print their geometries
for i in range(features_list.size().getInfo()):
    feature = ee.Feature(features_list.get(i))
    feature_id = feature.get('id').getInfo()  # Assuming 'id' is a property in your features
    geometry = feature.geometry().getInfo()
    
    print(f'Feature ID: {feature_id}')
    print('Geometry:', geometry)


Feature ID: None
Geometry: {'type': 'Polygon', 'coordinates': [[[35.146130655651405, 1.2707841895954126], [35.14685302368651, 1.2688092901137216], [35.14799010288401, 1.2674451401591884], [35.14916727820489, 1.26735153924897], [35.15277019739734, 1.264605386720509], [35.157991871304915, 1.2606288624618498], [35.165282452004256, 1.255444161293662], [35.172898649527, 1.2530947660470813], [35.1783208915973, 1.255961292852631], [35.180314086588666, 1.2578247689459106], [35.18620011241848, 1.2567280872655118], [35.19024898248702, 1.257169387076774], [35.19138161086418, 1.2588010279671342], [35.194409286235775, 1.2605887249405323], [35.1971338397432, 1.258591531369334], [35.202533787270845, 1.256420432147167], [35.20421930676472, 1.256799417555865], [35.20617238347019, 1.256340238689179], [35.20752349319732, 1.2570133905983987], [35.20861152640876, 1.258475639072616], [35.209784322186124, 1.2585201634077419], [35.21239290737897, 1.2588188832503349], [35.21712393992468, 1.2578158193389413], [

In [46]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define the asset ID of the FeatureCollection containing your polygon
fc_asset_id = 'users/zebrinasamy/GospicParcelsOne'

# Load the FeatureCollection
fc = ee.FeatureCollection(fc_asset_id)

# Define the ID of the feature you want to select
feature_id = 12345  # Replace with the desired feature ID

# Filter the FeatureCollection based on the "id" property
filtered_fc = fc.filter(ee.Filter.eq('id', 3560414))

# Get the first feature from the filtered FeatureCollection
polygon = filtered_fc.first()

# Get the geometry of the polygon
geometry = polygon.geometry()

# Define date range
start_date = '2022-04-01'
end_date = '2022-10-30'

# Load Sentinel-2 image collection
s2 = ee.ImageCollection('COPERNICUS/S2')
filtered_s2 = s2.filterBounds(geometry)\
                 .filterDate(start_date, end_date)\
                 .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 15)

# Clip images to the polygon
clipped_s2 = filtered_s2.map(lambda image: image.clip(geometry))

# Print the number of images in the filtered and clipped collection
print("Number of images:", clipped_s2.size().getInfo())

# Calculate NDVI for each image in the collection
def calculate_ndvi(image):
    return image.normalizedDifference(['B8', 'B4']).rename('NDVI')

ndvi_collection = clipped_s2.map(calculate_ndvi)

# Calculate median NDVI image
median_ndvi_image = ndvi_collection.median()

# Visualize median NDVI
Map = geemap.Map()
Map.centerObject(geometry, zoom=17)
Map.addLayer(median_ndvi_image, {'min': -1, 'max': 1, 'palette': ['yellow', 'orange', 'red']}, 'Median NDVI')  # Adjust palette here
#Map.addLayer(geometry, {}, 'Polygon')
Map.addLayerControl()
Map


Number of images: 30


Map(center=[44.42044926592382, 15.582487187574698], controls=(WidgetControl(options=['position', 'transparent_…

In [47]:
task = ee.batch.Export.image.toDrive(image=median_ndvi_image,
                                     description='median_ndvi_image',
                                     scale=0.005,
                                     region=geometry,
                                     fileNamePrefix='my_export_NDVI1',
                                     crs='EPSG:4326',
                                     folder='Croatia',
                                     fileFormat='GeoTIFF')
task.start()

In [8]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define the asset ID of the FeatureCollection containing your polygon
fc_asset_id = 'projects/ee-chesangirine55/assets/kamatiraboundary'

# Load the FeatureCollection
fc = ee.FeatureCollection(fc_asset_id)

# Define the ID of the feature you want to select
#feature_id = 12345  # Replace with the desired feature ID

# Filter the FeatureCollection based on the "id" property
#filtered_fc = fc.filter(ee.Filter.eq('id', 3560414))

# Get the first feature from the filtered FeatureCollection
polygon = fc.first()

# Get the geometry of the polygon
geometry = polygon.geometry()

# Define date range
start_date = '2022-04-01'
end_date = '2022-10-30'

# Load Sentinel-2 image collection
s2 = ee.ImageCollection('COPERNICUS/S2')
filtered_s2 = s2.filterBounds(geometry)\
                 .filterDate(start_date, end_date)\
                 .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 15)

# Clip images to the polygon
clipped_s2 = filtered_s2.map(lambda image: image.clip(geometry))

# Print the number of images in the filtered and clipped collection
print("Number of images:", clipped_s2.size().getInfo())

# Calculate NDVI for each image in the collection
def calculate_ndvi(image):
    return image.normalizedDifference(['B8', 'B4']).rename('NDVI')

ndvi_collection = clipped_s2.map(calculate_ndvi)

# Calculate median NDVI image
median_ndvi_image = ndvi_collection.median()

# Visualize median NDVI
Map = geemap.Map()
Map.centerObject(geometry, zoom=13)
Map.addLayer(median_ndvi_image, {'min': -1, 'max': 1, 'palette': ['yellow', 'orange', 'red']}, 'Median NDVI')  # Adjust palette here
#Map.addLayer(geometry, {}, 'Polygon')
Map.addLayerControl()
Map

Number of images: 17


Map(center=[1.2690236448652632, 35.192631821862136], controls=(WidgetControl(options=['position', 'transparent…

In [54]:
task = ee.batch.Export.image.toDrive(image=median_ndvi_image,
                                     description='median_ndvi_image',
                                     scale=100,
                                     region=geometry,
                                     fileNamePrefix='Zagreb_export_NDVI',
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')
task.start()

In [10]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define the asset ID of the FeatureCollection containing your polygons
fc_asset_id = 'projects/ee-chesangirine55/assets/kamatiraboundary'

# Load the FeatureCollection
fc = ee.FeatureCollection(fc_asset_id)

# Define date range
start_date = '2022-04-01'
end_date = '2022-10-30'

# Load Sentinel-2 image collection
s2 = ee.ImageCollection('COPERNICUS/S2')\
        .filterDate(start_date, end_date)\
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 15)

# Calculate NDVI for each image in the collection
def calculate_ndvi(image):
    return image.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Initialize the map
Map = geemap.Map()
Map.centerObject(fc, zoom=10)

# Iterate over each feature in the FeatureCollection
def process_feature(feature):
    geometry = feature.geometry()
    
    # Clip images to the polygon
    clipped_s2 = s2.filterBounds(geometry).map(lambda image: image.clip(geometry))
    
    # Calculate NDVI for each image
    ndvi_collection = clipped_s2.map(calculate_ndvi)
    
    # Calculate median NDVI image
    median_ndvi_image = ndvi_collection.median()
    
    # Visualize median NDVI for this feature
    Map.addLayer(median_ndvi_image, {'min': -1, 'max': 1, 'palette': ['yellow', 'orange', 'red']}, 'Median NDVI')
    # Uncomment to add the geometry layer for visualization
    # Map.addLayer(geometry, {}, 'Polygon')

# Get the list of features from the FeatureCollection as a client-side list
features_list = fc.toList(fc.size())

# Iterate over the features and process each one
for i in range(features_list.size().getInfo()):
    feature = ee.Feature(features_list.get(i))
    process_feature(feature)

# Add layer control and display the map
Map.addLayerControl()
Map


Map(center=[0.07598663852423793, 35.722413130991086], controls=(WidgetControl(options=['position', 'transparen…

In [9]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define the asset ID of the FeatureCollection containing your polygons
fc_asset_id = 'projects/ee-chesangirine55/assets/kamatiraboundary'

# Load the FeatureCollection
fc = ee.FeatureCollection(fc_asset_id)

# Define date range
start_date = '2022-04-01'
end_date = '2022-10-30'

# Load Sentinel-2 image collection
s2 = ee.ImageCollection('COPERNICUS/S2')\
        .filterDate(start_date, end_date)\
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 15)

# Calculate NDVI for each image in the collection
def calculate_ndvi(image):
    return image.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Initialize the map
Map = geemap.Map()
Map.centerObject(fc, zoom=10)

# Get the list of features from the FeatureCollection as a client-side list
features_list = fc.toList(fc.size())

# Iterate over the features and process each one
for i in range(features_list.size().getInfo()):
    feature = ee.Feature(features_list.get(i))
    geometry = feature.geometry()

    # Clip images to the polygon
    clipped_s2 = s2.filterBounds(geometry).map(lambda image: image.clip(geometry))

    # Calculate NDVI for each image
    ndvi_collection = clipped_s2.map(calculate_ndvi)

    # Calculate median NDVI image
    median_ndvi_image = ndvi_collection.median()

    # Add the median NDVI layer to the map for this feature
    Map.addLayer(median_ndvi_image, {'min': -1, 'max': 1, 'palette': ['yellow', 'orange', 'red']}, f'Median NDVI for feature {i+1}')
    # Uncomment to add the geometry layer for visualization
    # Map.addLayer(geometry, {}, f'Polygon {i+1}')

# Add layer control and display the map
Map.addLayerControl()
Map


Map(center=[1.269023644853747, 35.19263182187535], controls=(WidgetControl(options=['position', 'transparent_b…

In [10]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define the asset ID of the FeatureCollection containing your polygons
fc_asset_id = 'projects/ee-chesangirine55/assets/kamatiraboundary'

# Load the FeatureCollection
fc = ee.FeatureCollection(fc_asset_id)

# Define date range
start_date = '2022-04-01'
end_date = '2022-10-30'

# Load Sentinel-2 image collection
s2 = ee.ImageCollection('COPERNICUS/S2')\
        .filterDate(start_date, end_date)\
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 15)

# Calculate NDVI for each image in the collection
def calculate_ndvi(image):
    return image.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Initialize the map
Map = geemap.Map()
Map.centerObject(fc, zoom=10)

# Get the list of features from the FeatureCollection as a client-side list
features_list = fc.toList(fc.size())

# Iterate over the features and process each one
for i in range(features_list.size().getInfo()):
    feature = ee.Feature(features_list.get(i))
    geometry = feature.geometry()

    # Clip images to the polygon
    clipped_s2 = s2.filterBounds(geometry).map(lambda image: image.clip(geometry))

    # Calculate NDVI for each image
    ndvi_collection = clipped_s2.map(calculate_ndvi)

    # Calculate median NDVI image
    median_ndvi_image = ndvi_collection.median()

    # Add the median NDVI layer to the map for this feature
    Map.addLayer(median_ndvi_image, {'min': -1, 'max': 1, 'palette': ['yellow', 'orange', 'red']}, f'Median NDVI for feature {i+1}')
    # Uncomment to add the geometry layer for visualization
    # Map.addLayer(geometry, {}, f'Polygon {i+1}')

    # Export median NDVI image to Google Drive
    task = ee.batch.Export.image.toDrive(image=median_ndvi_image,
                                         description=f'median_ndvi_image_{i+1}',
                                         scale=1,
                                         region=geometry,
                                         fileNamePrefix=f'Farm_export_NDVI_{i+1}',
                                         crs='EPSG:4326',
                                         folder='NDVI',
                                         fileFormat='GeoTIFF')
    task.start()

# Add layer control and display the map
Map.addLayerControl()
Map


Map(center=[1.269023644853747, 35.19263182187535], controls=(WidgetControl(options=['position', 'transparent_b…

In [17]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Define the asset ID of the FeatureCollection containing your polygons
fc_asset_id = 'projects/serene-bastion-406504/assets/Merged_Plots'

# Load the FeatureCollection
fc = ee.FeatureCollection(fc_asset_id)

# Define date range for March and April 2023
start_date_march = '2023-03-01'
end_date_march = '2023-03-31'

start_date_april = '2023-04-01'
end_date_april = '2023-04-30'

# Load Sentinel-2 image collection for March and April 2023
s2_march = ee.ImageCollection('COPERNICUS/S2_SR')\
            .filterDate(start_date_march, end_date_march)\
            .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 15)

s2_april = ee.ImageCollection('COPERNICUS/S2_SR')\
            .filterDate(start_date_april, end_date_april)\
            .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 15)

# Calculate NDVI for each image in the collection
def calculate_ndvi(image):
    return image.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Initialize the map
Map = geemap.Map()
Map.centerObject(fc, zoom=16)

# Get the list of features from the FeatureCollection as a client-side list
features_list = fc.toList(fc.size())

# Function to add NDVI layers to the map
def add_ndvi_layers(image_collection, month_name):
    # Iterate over the features and process each one
    for i in range(features_list.size().getInfo()):
        feature = ee.Feature(features_list.get(i))
        geometry = feature.geometry()

        # Clip images to the polygon
        clipped_s2 = image_collection.filterBounds(geometry).map(lambda image: image.clip(geometry))

        # Calculate NDVI for each image
        ndvi_collection = clipped_s2.map(calculate_ndvi)

        # Calculate median NDVI image
        median_ndvi_image = ndvi_collection.median()

        # Add the median NDVI layer to the map for this feature
        Map.addLayer(median_ndvi_image, {'min': -1, 'max': 1, 'palette': ['yellow', 'orange', 'red']}, f'{month_name} NDVI for feature {i+1}')
        # Uncomment to add the geometry layer for visualization
        # Map.addLayer(geometry, {}, f'Polygon {i+1}')

# Add NDVI layers for March and April
add_ndvi_layers(s2_march, 'March')
add_ndvi_layers(s2_april, 'April')

# Add layer control and display the map
Map.addLayerControl()
Map


Map(center=[0.07598663852423793, 35.722413130991086], controls=(WidgetControl(options=['position', 'transparen…

In [11]:
import ee
import geemap
import pandas as pd

# Initialize Earth Engine
ee.Initialize()

# Define the asset ID of the FeatureCollection containing your polygons
fc_asset_id = 'projects/ee-chesangirine55/assets/kamatiraboundary'

# Load the FeatureCollection
fc = ee.FeatureCollection(fc_asset_id)

# Define date range for March and April 2023
start_date_march = '2023-03-01'
end_date_march = '2023-03-31'

start_date_april = '2023-04-01'
end_date_april = '2023-04-30'

# Load Sentinel-2 image collection for March and April 2023
s2_march = ee.ImageCollection('COPERNICUS/S2_SR')\
            .filterDate(start_date_march, end_date_march)\
            .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 15)

s2_april = ee.ImageCollection('COPERNICUS/S2_SR')\
            .filterDate(start_date_april, end_date_april)\
            .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 15)

# Function to sample random points and append NDVI values
def sample_and_export_ndvi(feature, image_collection, month_name):
    plot_id = feature.get('Plot_No').getInfo()  # Assuming 'Plot_No' is the property containing plot IDs
    geometry = feature.geometry()

    # Sample random points within the plot geometry
    random_points = ee.FeatureCollection.randomPoints(geometry, 200)

    # Clip images to the plot geometry
    clipped_s2 = image_collection.filterBounds(geometry).map(lambda image: image.clip(geometry))

    # Calculate NDVI for each image at sampled points
    def calculate_ndvi_at_point(feature):
        try:
            # Get the first image in the collection
            image = ee.Image(clipped_s2.first())
            
            # Check if the image has the required bands
            if not image.bandNames().contains('B8') or not image.bandNames().contains('B4'):
                raise ValueError("Image does not contain required bands 'B8' and 'B4'")
            
            # Calculate NDVI
            ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
            
            # Reduce region to get NDVI value at each point
            return feature.set('NDVI', ndvi.reduceRegion(reducer=ee.Reducer.first(), geometry=feature.geometry(), scale=10).get('NDVI'))
        except ee.EEException as e:
            print(f'Error calculating NDVI for Plot {plot_id}: {str(e)}')
            return feature.set('NDVI', None)
        except ValueError as e:
            print(f'Error: {str(e)}')
            return feature.set('NDVI', None)

    ndvi_points = random_points.map(calculate_ndvi_at_point)

    # Prepare data for CSV
    data = []

    # Iterate over sampled points
    for point in ndvi_points.getInfo()['features']:
        coords = point['geometry']['coordinates']
        ndvi_value = point['properties']['NDVI']
        
        # Check if NDVI value exists
        if ndvi_value is not None:
            data.append({
                'Longitude': coords[0],
                'Latitude': coords[1],
                'NDVI': ndvi_value,
                'Plot_ID': plot_id,
                'Month': month_name
            })

    # Create DataFrame
    df = pd.DataFrame(data)

    # Export to CSV
    csv_filename = f'{plot_id}_{month_name}_sampled_points_with_NDVI.csv'
    df.to_csv(csv_filename, index=False)
    print(f'Exported {csv_filename}')

# Sample and export NDVI values for March and April
for feature in fc.toList(fc.size()).getInfo():
    sample_and_export_ndvi(ee.Feature(feature), s2_march, 'March')
    sample_and_export_ndvi(ee.Feature(feature), s2_april, 'April')


EEException: Image.normalizedDifference: Parameter 'input' is required and may not be null.